In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

n

300

In [3]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

,n1,n2,e,w
0,1,2,0,1
1,1,3,0,2
2,1,4,0,6
3,1,5,0,1
4,1,6,0,3
...,...,...,...,...
44845,297,299,0,1
44846,297,300,0,1
44847,298,299,0,4
44848,298,300,0,2


In [4]:
# create nodes data frame
nodes_from = df.loc[df["e"]==1][["n1","w"]].groupby(['n1']).sum()
nodes_from
nodes_to = df.loc[df["e"]==1][["n2","w"]].groupby(['n2']).sum()
nodes_to

nodes = nodes_from.join(nodes_to, lsuffix='_from', rsuffix='_to', how = 'outer')
nodes['node_impact'] = nodes.w_from.fillna(0) + nodes.w_to.fillna(0)
nodes = nodes.drop(columns=['w_from', 'w_to'])
nodes['current_degree'] = 0
nodes['splex'] = nodes.index
nodes = nodes.reset_index().rename(columns={"index":"node_number"})
nodes

,node_number,node_impact,current_degree,splex
0,1,431.0,0,1
1,2,115.0,0,2
2,3,46.0,0,3
3,4,113.0,0,4
4,5,111.0,0,5
...,...,...,...,...
295,296,6.0,0,296
296,297,165.0,0,297
297,298,50.0,0,298
298,299,385.0,0,299


In [5]:
edges = df.copy()
edges['w'] = edges['w'] * (1-(edges['e']*2))
edges['e'] = 0
edges.loc[edges['w'] < 0]

,n1,n2,e,w
8,1,10,0,-5
12,1,14,0,-1
13,1,15,0,-3
14,1,16,0,-1
16,1,18,0,-2
...,...,...,...,...
44827,293,299,0,-1
44828,293,300,0,-3
44831,294,297,0,-1
44833,294,299,0,-2


In [6]:
def is_splex(nodes, plex_number, s) -> bool | pd.DataFrame:
    splex = nodes.loc[nodes["splex"] == plex_number]
    min_degree = len(splex.index) - s
    problem_nodes = splex.loc[splex["current_degree"] < min_degree]
    
    if len(problem_nodes.index) == 0:
        return True
    else:
        return problem_nodes

is_splex(nodes, 2, 1)

True

In [7]:
existing_edges = df.loc[df["e"]==1].sort_values("w", ascending=False)
existing_edges

,n1,n2,e,w
17490,66,202,1,13
39562,197,266,1,12
28232,118,154,1,12
43801,254,287,1,12
22823,90,219,1,12
...,...,...,...,...
4335,15,256,1,1
7217,26,69,1,1
37709,180,300,1,1
37716,181,188,1,1


In [8]:
nodes_in_plex = nodes.loc[(nodes["splex"]==201) | (nodes["splex"] ==30), "node_number"].values
nodes_in_plex
#for node in nodes_in_plex:
#    print(nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]<3)

array([ 30, 201], dtype=int64)

In [9]:
existing_edges = df.loc[df["e"]==1].sort_values("w", ascending=False)

remaining_edges = len(existing_edges) # wozu?
start = time.time()
for index, row in existing_edges.iterrows():
    # get plex assignment of both nodes
    n1_plex = nodes.loc[nodes["node_number"]==row["n1"], "splex"].values[0]
    n2_plex = nodes.loc[nodes["node_number"]==row["n2"], "splex"].values[0]
    # all nodes that would be in the plex if we merged it
    nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
    # I guess this is for the TODO
    other_edges_to_add = existing_edges.loc[(existing_edges["n1"].isin(nodes_in_plex)) & 
                                            (existing_edges["n2"].isin(nodes_in_plex))]
    
    
    number_of_nodes = len(nodes_in_plex)
    edges_missing = False

    for node in nodes_in_plex:
        # check if it would be a valid splex if we merge
        if nodes.loc[nodes["node_number"]==node,"current_degree"].values[0] < (number_of_nodes - s):
            edges_missing = True
            # TODO: Add other edges
    
    # if it would be a valid splex, actually merge it
    if not edges_missing:
        # merge them
        nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
        # include the connectig edge
        edges.loc[(edges["n1"]==row["n1"]) & (edges["n2"]==row["n2"]), "e"] = 1 
        # update node info (degree and node impact)
        nodes.loc[nodes["node_number"]==row["n1"], "current_degree"] +=1
        nodes.loc[nodes["node_number"]==row["n2"], "current_degree"] +=1
        nodes.loc[nodes["node_number"]==row["n1"], "node_impact"] -=row["w"]
        nodes.loc[nodes["node_number"]==row["n2"], "node_impact"] -=row["w"]

        remaining_edges -= 1 #wozu?
print(round(time.time()-start, 2), "seconds")

40.41 seconds


In [10]:
nodes

,node_number,node_impact,current_degree,splex
0,1,420.0,2,1
1,2,96.0,2,2
2,3,34.0,3,95
3,4,100.0,2,4
4,5,97.0,3,5
...,...,...,...,...
295,296,3.0,1,179
296,297,165.0,0,297
297,298,43.0,1,19
298,299,363.0,3,177


In [13]:
edges.loc[edges['e'] >0]

,n1,n2,e,w
226,1,228,1,-6
230,1,232,1,-5
412,2,116,1,-8
502,2,206,1,-11
688,3,95,1,-1
...,...,...,...,...
43648,251,275,1,-8
43801,254,287,1,-12
44059,260,290,1,-2
44188,264,269,1,-5
